# Práctico 2-3: Árboles de decisión


Estos modelos pueden verse como un juego de preguntas. Para clasificar un dato se realizan una serie de preguntas; la respuesta a cada pregunta determina una bifurcación en un árbol (la primera pregunta corresponde a la raíz del árbol). Una bifurcación puede llevar a otra bifurcación (otra pregunta), o puede ser suficiente para determinar la clase, en cuyo caso se llega a una _hoja_.

Existen diferentes métodos para determinar la secuencia de preguntas que constituye el proceso de clasificación en un modelo de este tipo. Abajo vemos cómo ajustar un árbol para clasificar flores de iris. Una ventaja de este tipo de modelos es que suelen ser fácilmente interpretables por un humano, lo que siempre es deseable.


### Preparación del entorno



# Preámbulo

*   Importamos los paquetes en el código (ya instalados anteriormente)
*   Configuramos algunos parámetros de visualización (tamaño de letra, gráficas, et.c)
*   Cargamos datos para experimentar

In [ ]:

#
# importación
#
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt

import numpy as np
import numpy.random as rng

mpl.rcParams['figure.dpi'] = 100
mpl.rcParams['savefig.dpi'] = 150

mpl.rcParams['font.size'] = 10
mpl.rcParams['legend.fontsize'] = 'medium'
mpl.rcParams['figure.titlesize'] = 'medium'






## Datos: valor de diamantes

Los árboles de decisión son particularmente útiles cuando las variables sobre las cuales debe decidirse incluyen datos _categóricos_, que no pueden considerarse numéricos y por ende no tiene sentido tomar como entrada en una regresión.

En este problema se busca predecir el precio de diamantes a partir de un conjunto de características.
Los datos los bajamos directamente de la red y los leemos como un archivo CSV.


In [ ]:
import urllib
import csv

urllib.request.urlretrieve('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diamond.csv', 'diamond.csv')
with open('diamond.csv') as f:
    reader = csv.reader(f)
    data = list(reader)

header = data[0]
print(header)
y = np.array([float(row[-1]) for row in data[1:]]) # ultima columna es el precio
X_quilates = np.array([float(row[0]) for row in data[1:]]) # primera columna son los kilates; ya es un número
X_corte    = [row[1] for row in data[1:]] # segunda columna es el corte
X_color    = [row[2] for row in data[1:]] # tercera columna es el color
X_claridad = [row[3] for row in data[1:]] # cuarta columna es la claridad
X_pulido   = [row[4] for row in data[1:]] # quinta columna es el pulido
X_simetria = [row[5] for row in data[1:]] # sexta columna es la simetría
X_reporte  = [row[6] for row in data[1:]] # séptima columna es el reporte, que no tengo idea qué es
#
# categorías de cada columna
#
cortes     = np.unique(X_corte)
colores    = np.unique(X_color)
claridades = np.unique(X_claridad)
pulidos    = np.unique(X_pulido)
simetrias  = np.unique(X_simetria)
reportes   = np.unique(X_reporte)
#
# reemplazamos las categorías por números (índices) 
#
X_corte    = np.array([np.where(cortes == corte)[0][0] for corte in X_corte])
X_color    = np.array([np.where(colores == color)[0][0] for color in X_color])
X_claridad = np.array([np.where(claridades == claridad)[0][0] for claridad in X_claridad])
X_pulido   = np.array([np.where(pulidos == pulido)[0][0] for pulido in X_pulido])
X_simetria = np.array([np.where(simetrias == simetria)[0][0] for simetria in X_simetria])
X_reporte  = np.array([np.where(reportes == reporte)[0][0] for reporte in X_reporte])
X = np.concatenate([X_quilates[:, np.newaxis],
                    X_corte[:, np.newaxis], 
                    X_color[:, np.newaxis], 
                    X_claridad[:, np.newaxis], 
                    X_pulido[:, np.newaxis], 
                    X_simetria[:, np.newaxis], 
                    X_reporte[:, np.newaxis]], axis=1)
print(X.shape)
print(y.shape)




## Entrenamiento y validación

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#
# arboles de decisión
#
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

clf = DecisionTreeClassifier(random_state=42)
model = clf.fit(X, y)




## Evaluación


In [ ]:

from sklearn import metrics

y_test_pred = model.predict(X_test)
#
# evaluamos distintas métricas de desempeño
#
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
mae = metrics.mean_absolute_error(y_test, y_test_pred)
mape = metrics.mean_absolute_percentage_error(y_test, y_test_pred)
print(f"rmse: {rmse:.2f}, mae: {mae:.2f}, mape: {mape:.2f}")

idx = np.argsort(y_test)
y_test_sorted = y_test[idx]
y_test_pred_sorted = y_test_pred[idx]
plt.figure(figsize=(8,8))
plt.semilogy(y_test_sorted, label='real',lw=1)
plt.semilogy(y_test_pred_sorted, label='predicción',lw=1)
plt.grid()
plt.ylabel('precio')
plt.xlabel('instancia (en orden de precio)')
plt.legend()
plt.show()

### Observaciones

Como puede verse, el ajuste es bastante bueno. Como observación interesante, el modelo aprendido tiende a sub-valorar el precio real.
Esto lo podemos ver también en el histograma del error de predicción siguiente:

## Visualización

Hay varias formas de visualizar árboles. En orden de sofisticación:

* Funciones de Sklearn (ya seguramente instaladas en su entorno)
* Biblioteca `graphviz`
* `dtreeviz`

Veamos cada una de ellas. Como implican instalar paquetes es posible que algo falle, por lo que separamos las celdas según el tipo y ejecutamos las que podamos.



### SKlearn

La biblioteca Sklearn ya debe estar instalada en su sistema, ya que muchas de las funciones que usamos antes son de ahí.
Si bien el resultado no es muy bonito, siempre está disponible, lo cual no es menor.

In [ ]:
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(clf,
                   feature_names=iris.feature_names,
                   class_names=iris.target_names,
                   filled=True)

fig.savefig("decistion_tree_sklearn.png")

#
# visualización con dtreeviz
#
# from dtreeviz.trees import dtreeviz # remember to load the package



### Graphviz

Graphviz es una biblioteca de dibujo muy completa y muy conocida para muchos tipos de datos.

In [ ]:

import graphviz
dot_data = tree.export_graphviz(clf, out_file=None,
                                feature_names=iris.feature_names,
                                class_names=iris.target_names,
                                filled=True)

graph = graphviz.Source(dot_data, format="png")
graph.render("decision_tree_graphivz")


### Dtreeviz

Esta biblioteca es específica para dibujar árboles de decisión.

In [ ]:
#!pip install --quiet dtreeviz
!conda config --add channels defaults
!conda install -c defaults dtreeviz

import dtreeviz
from IPython.display import SVG

viz_model = dtreeviz.model(clf, X, y,
                target_name="target",
                feature_names=iris.feature_names,
                class_names=list(iris.target_names))

viz = viz_model.view()     # render as SVG into internal object

viz.save("decision_tree_dtreeviz.svg")

SVG("decision_tree_dtreeviz.svg")